# Modelo para classificação de widgets do tipo Dropdown e Suggestion Box

__Importando bibliotecas__

In [1]:
# https://docs.python.org/3/library/random.html
import random

# funções e operações para cálculos numéricos / https://docs.scipy.org/doc/numpy/reference/
import numpy as np
print("numpy version: {}". format(np.__version__))

# manipulação e processamento de dados / https://pandas.pydata.org/pandas-docs/stable/
import pandas as pd
print("pandas version: {}". format(pd.__version__))

# pré processamento de dados / https://scikit-learn.org/stable/modules/preprocessing.html
import sklearn.preprocessing as preprocessing

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

numpy version: 1.16.4
pandas version: 0.25.0


In [2]:
path_file1 = 'DataSets/widgets/5-test-widget-com-url.tab'
file1 = pd.read_csv(path_file1, sep='\t')

path_file2 = 'DataSets/widgets/5-training-widget-com-url.tab'
file2 = pd.read_csv(path_file2, sep='\t')

frames = [file1, file2]
dataset = pd.concat(frames)

dataset = dataset[dataset.widgetClass != 'Discard']

dataset.head()

,positionLeft,positionTop,width,height,childCount,domHeight,domLevelMoreElements,domLevelElements,domWidth,mouseX,mouseY,numWords,numWordsChild,numLinks,url,event,widgetClass
0,1119,16,248,30,175,7,6,72,125,1267,87,32,1.33,25,https://www.google.com/,mouseover,Dropdown
1,1263,16,30,30,0,0,1,1,0,1273,44,0,0.00,0,https://www.google.com/,mouseover,Other
2,0,0,0,0,156,4,3,72,116,1273,44,30,1.36,21,https://www.google.com/,mouseover,Dropdown
3,0,0,0,0,0,0,1,1,0,1273,44,0,0.00,0,https://www.google.com/,mouseover,Other
4,1293,16,45,30,38,6,1,7,20,1273,44,33,2.36,9,https://www.google.com/,mouseover,Dropdown


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43347 entries, 0 to 31396
Data columns (total 17 columns):
positionLeft            43347 non-null int64
positionTop             43347 non-null int64
width                   43347 non-null int64
height                  43347 non-null int64
childCount              43347 non-null int64
domHeight               43347 non-null int64
domLevelMoreElements    43347 non-null int64
domLevelElements        43347 non-null int64
domWidth                43347 non-null int64
mouseX                  43347 non-null int64
mouseY                  43347 non-null int64
numWords                43347 non-null int64
numWordsChild           43347 non-null float64
numLinks                43347 non-null int64
url                     43347 non-null object
event                   43347 non-null object
widgetClass             43347 non-null object
dtypes: float64(1), int64(13), object(3)
memory usage: 6.0+ MB


In [4]:
dataset.describe()

,positionLeft,positionTop,width,height,childCount,domHeight,domLevelMoreElements,domLevelElements,domWidth,mouseX,mouseY,numWords,numWordsChild,numLinks
count,43347.000000,4.334700e+04,43347.000000,43347.000000,43347.000000,43347.000000,43347.000000,43347.000000,43347.000000,43347.000000,43347.000000,43347.000000,43347.000000,43347.000000
mean,-1152.049554,-8.698751e+02,269.255612,67.907006,13.469098,2.528687,1.520544,4.289916,6.708953,563.608600,126.382702,9.938566,1.916672,2.146769
std,38719.137384,3.876365e+04,342.143788,164.844927,27.460319,2.833679,1.311200,8.573024,15.004063,436.749093,209.362989,18.307429,2.632862,5.445756
min,-1000000.000000,-1.002171e+06,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,93.000000,3.000000,0.000000,0.000000,0.000000
50%,370.000000,1.660000e+02,139.000000,22.000000,2.000000,1.000000,1.000000,1.000000,1.000000,581.000000,30.000000,2.000000,1.070000,0.000000
75%,684.000000,5.380000e+02,440.000000,46.500000,11.000000,4.000000,1.000000,4.000000,5.000000,924.000000,195.000000,10.000000,2.810000,1.000000
max,2702.000000,1.991900e+04,6755.000000,3824.000000,235.000000,17.000000,12.000000,87.000000,125.000000,1367.000000,4306.000000,143.000000,50.000000,50.000000


In [5]:
dataset['widgetClass'].value_counts()

Other            33478
Dropdown          6459
Suggestionbox     3410
Name: widgetClass, dtype: int64

In [6]:
dataset.loc[(dataset['widgetClass'] == 'Dropdown'), 'widgetClass'] = 0
dataset.loc[(dataset['widgetClass'] == 'Other'), 'widgetClass'] = 1
dataset.loc[(dataset['widgetClass'] == 'Suggestionbox'), 'widgetClass'] = 2

dataset.rename(columns={'widgetClass': 'target'}, inplace=True)

dataset['target'].value_counts()

1    33478
0     6459
2     3410
Name: target, dtype: int64

In [7]:
dataset['event'].value_counts()

mouseover    18297
click        13691
none         10637
keyup          722
Name: event, dtype: int64

In [8]:
dataset.loc[(dataset['event'] == 'mouseover'), 'event'] = 0
dataset.loc[(dataset['event'] == 'click'), 'event'] = 1
dataset.loc[(dataset['event'] == 'none'), 'event'] = 2
dataset.loc[(dataset['event'] == 'keyup'), 'event'] = 3

dataset['event'].value_counts()

0    18297
1    13691
2    10637
3      722
Name: event, dtype: int64

In [9]:
dataset.rename(columns={'url': 'group'}, inplace=True)

In [10]:
from sklearn.model_selection import cross_validate, StratifiedKFold, GridSearchCV, train_test_split, GroupKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Grid Search for tuning parameters
from sklearn.model_selection import GridSearchCV
# RandomizedSearch for tuning (possibly faster than GridSearch)
from sklearn.model_selection import RandomizedSearchCV

In [11]:
class Classificador():
    def __init__ (self, seed):
        self._seed = seed

    def classificar(self, dataset, fold_method, n_splits_value, search, params, score_op, numero_cv):
        X = dataset.loc[:, ~dataset.columns.isin(['target', 'group'])]
        y = dataset['target'] ## seleciona o "rótulo" que será classificado
        y.columns = ['target']

        random.seed(self._seed) ## gera seed
        
        modelo = self.get_classificador(X, y, search, params, score_op, numero_cv)
        
        ## StratifiedKfold separa os folds preservando a porcentagem de amostras para cada classe do target ##
        ## GroupKfold separa os folds preservando grupos, sendo que os mesmo grupo não aparecerá em diferentes folds ##
        # n_splits = número de folds
        # random_state = seed
    
        if(fold_method == "StratifiedKfold"):  
            fold = StratifiedKFold(n_splits = n_splits_value, random_state = self._seed)
            scores = cross_validate(modelo, X, y, cv = fold, scoring = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'])
            
        elif(fold_method == "GroupKfold"):
            fold = GroupKFold(n_splits = n_splits_value)
            scores = cross_validate(modelo, X, y, cv = fold, groups = dataset['group'], scoring = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'])
        
        ## cv = método de cross validation
        ## scoring = métricas para avaliação do modelo
        ## f1_macro = calcula f1 (média harmônica de precision e recall) separado por classes, mas sem estabelecer pesos
        for i in range(n_splits_value):
            print("Fold %s - Accuracy: %.5s, Precision: %.5s, Recall: %.5s, F-measure: %.5s" % (i, scores['test_accuracy'][i], scores['test_precision_macro'][i], scores['test_recall_macro'][i], scores['test_f1_macro'][i]))
        
        print("Accuracy: %s on average and %s SD" % (scores['test_accuracy'].mean(), scores['test_accuracy'].std()))
        print("Precision: %s on average and %s SD" % (scores['test_precision_macro'].mean(), scores['test_precision_macro'].std()))
        print("Recall: %s on average and %s SD" % (scores['test_recall_macro'].mean(), scores['test_recall_macro'].std()))
        print("F-measure: %s on average and %s SD" % (scores['test_f1_macro'].mean(), scores['test_f1_macro'].std()))

        return 0
    
    def hypersearch(self, search, X, y, params, estimator, score_op, numero_cv):
        if search == "grid":
            clf = GridSearchCV(
                estimator = estimator, 
                param_grid = params, 
                scoring = score_op,
                cv = numero_cv,
                n_jobs = -1
            )
        elif search == "random":           
            clf = RandomizedSearchCV(
                estimator = estimator,
                param_distributions = params,
                scoring = score_op,
                cv = numero_cv,
                n_jobs = -1
            )
            
        clf.fit(X, y)
        print(clf.best_params_)
        
        return clf
    

In [12]:
class DecisionTree(Classificador):
    def __init__ (self, seed = 42):
        Classificador.__init__(self, seed)
        self.tipo = 'decision_tree'

    def get_classificador(self, X, y, search, params, score_op, numero_cv):
        print('-- DECISION TREE --')
        if(search != None):
            estimator = DecisionTreeClassifier()
            clf = Classificador.hypersearch(self, search, X, y, params, estimator, score_op, numero_cv)
            model = DecisionTreeClassifier(random_state = self._seed)
            model.set_params(**clf.best_params_)
        else:
            model = DecisionTreeClassifier(random_state = self._seed)
        return model

class KNN(Classificador):
    def __init__ (self, seed = 42):
        Classificador.__init__(self, seed)
        self.tipo = 'decision_tree'

    def get_classificador(self, X, y):
        print('-- KNN --')
        model = KNeighborsClassifier()
        return model

class SVM(Classificador):
    def __init__ (self, seed = 42):
        Classificador.__init__(self, seed)
        self.tipo = 'decision_tree'

    def get_classificador(self, X, y):
        print('-- SVM --')
        model = SVC(random_state=self._seed, gamma = 'scale')
        return model

class RandomForest(Classificador):
    def __init__ (self, seed = 42):
        Classificador.__init__(self, seed)
        self.tipo = 'decision_tree'

    def get_classificador(self, X, y):
        print('-- RANDOM FOREST --')
        model = RandomForestClassifier(random_state=self._seed, n_estimators = 100)
        return model

class LR(Classificador):
    def __init__ (self, seed = 42):
        Classificador.__init__(self, seed)
        self.tipo = 'decision_tree'

    def get_classificador(self, X, y):
        print('-- LOGISTIC REGRESSION --')
        model = LogisticRegression(random_state=self._seed, multi_class = 'auto', solver = 'lbfgs')
        return model

Paramêtros do Decision Tree (['class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'presort', 'random_state', 'splitter'])

In [29]:
classificador = DecisionTree(42)
classificador.classificar(
    dataset, 
    fold_method = "GroupKfold", 
    n_splits_value = 10, 
    search = 'grid', 
    params = {
        'criterion': ["gini", "entropy"],
        'max_depth': [10, 50, 100, None],
        'min_samples_split': [0.1, 2, 10, 100],
        'min_samples_leaf': [0.1, 2, 10, 100],
        'class_weight': [None, 'balanced']
    }, 
    score_op = 'recall_macro', 
    numero_cv = 5);

-- DECISION TREE --
{'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'min_samples_leaf': 2, 'min_samples_split': 2}
Fold 0 - Accuracy: 0.991, Precision: 0.977, Recall: 0.996, F-measure: 0.986
Fold 1 - Accuracy: 0.985, Precision: 0.973, Recall: 0.890, F-measure: 0.924
Fold 2 - Accuracy: 0.991, Precision: 0.996, Recall: 0.958, F-measure: 0.976
Fold 3 - Accuracy: 0.991, Precision: 0.979, Recall: 0.996, F-measure: 0.987
Fold 4 - Accuracy: 0.993, Precision: 0.962, Recall: 0.829, F-measure: 0.867
Fold 5 - Accuracy: 0.986, Precision: 0.967, Recall: 0.983, F-measure: 0.974
Fold 6 - Accuracy: 0.976, Precision: 0.927, Recall: 0.869, F-measure: 0.896
Fold 7 - Accuracy: 0.947, Precision: 0.940, Recall: 0.936, F-measure: 0.934
Fold 8 - Accuracy: 0.950, Precision: 0.941, Recall: 0.917, F-measure: 0.928
Fold 9 - Accuracy: 0.871, Precision: 0.734, Recall: 0.870, F-measure: 0.759
Accuracy: 0.9686126699152634 on average and 0.036296694110174776 SD
Precision: 0.9403266085481242 on average and

In [30]:
classificador = DecisionTree(42)
classificador.classificar(
    dataset, 
    fold_method = "GroupKfold", 
    n_splits_value = 10, 
    search = 'random', 
    params = {
        'criterion': ["gini", "entropy"],
        'max_depth': [10, 50, 100, None],
        'min_samples_split': [0.1, 2, 10, 100],
        'min_samples_leaf': [0.1, 2, 10, 100],
        'class_weight': [None, 'balanced']
    }, 
    score_op = 'recall_macro', 
    numero_cv = 5);

-- DECISION TREE --
{'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': None, 'criterion': 'gini', 'class_weight': 'balanced'}
Fold 0 - Accuracy: 0.991, Precision: 0.977, Recall: 0.996, F-measure: 0.986
Fold 1 - Accuracy: 0.988, Precision: 0.968, Recall: 0.932, F-measure: 0.948
Fold 2 - Accuracy: 0.991, Precision: 0.996, Recall: 0.958, F-measure: 0.976
Fold 3 - Accuracy: 0.991, Precision: 0.979, Recall: 0.996, F-measure: 0.987
Fold 4 - Accuracy: 0.995, Precision: 0.974, Recall: 0.829, F-measure: 0.873
Fold 5 - Accuracy: 0.984, Precision: 0.970, Recall: 0.964, F-measure: 0.967
Fold 6 - Accuracy: 0.972, Precision: 0.903, Recall: 0.868, F-measure: 0.884
Fold 7 - Accuracy: 0.946, Precision: 0.939, Recall: 0.936, F-measure: 0.934
Fold 8 - Accuracy: 0.982, Precision: 0.968, Recall: 0.936, F-measure: 0.951
Fold 9 - Accuracy: 0.892, Precision: 0.783, Recall: 0.815, F-measure: 0.794
Accuracy: 0.9738115693323225 on average and 0.030252299228046543 SD
Precision: 0.9461435835721437 on av

In [15]:
classificador = DecisionTree(42)
classificador.classificar(
    dataset, 
    fold_method = "GroupKfold", 
    n_splits_value = 10, 
    search = None, 
    params = None, 
    score_op = None, 
    numero_cv = None);

-- DECISION TREE --
Fold 0 - Accuracy: 0.991, Precision: 0.977, Recall: 0.996, F-measure: 0.986
Fold 1 - Accuracy: 0.985, Precision: 0.973, Recall: 0.890, F-measure: 0.924
Fold 2 - Accuracy: 0.991, Precision: 0.996, Recall: 0.958, F-measure: 0.976
Fold 3 - Accuracy: 0.991, Precision: 0.979, Recall: 0.996, F-measure: 0.987
Fold 4 - Accuracy: 0.995, Precision: 0.974, Recall: 0.829, F-measure: 0.873
Fold 5 - Accuracy: 0.986, Precision: 0.967, Recall: 0.983, F-measure: 0.974
Fold 6 - Accuracy: 0.972, Precision: 0.863, Recall: 0.841, F-measure: 0.846
Fold 7 - Accuracy: 0.947, Precision: 0.940, Recall: 0.936, F-measure: 0.934
Fold 8 - Accuracy: 0.947, Precision: 0.940, Recall: 0.916, F-measure: 0.926
Fold 9 - Accuracy: 0.857, Precision: 0.706, Recall: 0.838, F-measure: 0.740
Accuracy: 0.9667591672866089 on average and 0.0402105793693622 SD
Precision: 0.9320122020328473 on average and 0.08325149210617887 SD
Recall: 0.9188337048415314 on average and 0.06274963527892059 SD
F-measure: 0.91714006

In [16]:
classificador = DecisionTree(42)
classificador.classificar(
    dataset, 
    fold_method = "StratifiedKfold", 
    n_splits_value = 10, 
    search = 'grid', 
    params = {
        'criterion': ["gini", "entropy"],
        'max_depth': [10, 50, 100, None],
        'min_samples_split': [0.1, 2, 10, 100],
        'min_samples_leaf': [0.1, 2, 10, 100],
        'class_weight': [None, 'balanced']
    }, 
    score_op = 'recall_macro', 
    numero_cv = 5);

-- DECISION TREE --
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 10}
Fold 0 - Accuracy: 0.999, Precision: 0.998, Recall: 0.999, F-measure: 0.999
Fold 1 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 2 - Accuracy: 0.996, Precision: 0.991, Recall: 0.998, F-measure: 0.995
Fold 3 - Accuracy: 0.996, Precision: 0.985, Recall: 0.998, F-measure: 0.991
Fold 4 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 5 - Accuracy: 0.986, Precision: 0.950, Recall: 0.994, F-measure: 0.970
Fold 6 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 7 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 8 - Accuracy: 0.998, Precision: 0.995, Recall: 0.999, F-measure: 0.997
Fold 9 - Accuracy: 0.999, Precision: 0.999, Recall: 0.999, F-measure: 0.999
Accuracy: 0.9976008694937984 on average and 0.004064419338288494 SD
Precision: 0.9922215836681275 on average and 0.014525783458225356 SD

In [17]:
classificador = DecisionTree(42)
classificador.classificar(
    dataset, 
    fold_method = "StratifiedKfold", 
    n_splits_value = 10, 
    search = 'random', 
    params = {
        'criterion': ["gini", "entropy"],
        'max_depth': [10, 50, 100, None],
        'min_samples_split': [0.1, 2, 10, 100],
        'min_samples_leaf': [0.1, 2, 10, 100],
        'class_weight': [None, 'balanced']
    }, 
    score_op = 'recall_macro', 
    numero_cv = 5);

-- DECISION TREE --
{'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 10, 'criterion': 'gini', 'class_weight': 'balanced'}
Fold 0 - Accuracy: 0.999, Precision: 0.998, Recall: 0.999, F-measure: 0.999
Fold 1 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 2 - Accuracy: 0.996, Precision: 0.991, Recall: 0.998, F-measure: 0.995
Fold 3 - Accuracy: 0.996, Precision: 0.985, Recall: 0.998, F-measure: 0.991
Fold 4 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 5 - Accuracy: 0.988, Precision: 0.959, Recall: 0.995, F-measure: 0.976
Fold 6 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 7 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 8 - Accuracy: 0.998, Precision: 0.995, Recall: 0.999, F-measure: 0.997
Fold 9 - Accuracy: 0.999, Precision: 0.999, Recall: 0.999, F-measure: 0.999
Accuracy: 0.9978776861027949 on average and 0.0032976728633234476 SD
Precision: 0.9931139910088976 on average and 0.012019443360355465 SD


In [19]:
classificador = DecisionTree(42)
classificador.classificar(
    dataset, 
    fold_method = "StratifiedKfold", 
    n_splits_value = 10, 
    search = None, 
    params = None, 
    score_op = None, 
    numero_cv = None);

-- DECISION TREE --
Fold 0 - Accuracy: 0.999, Precision: 0.997, Recall: 0.999, F-measure: 0.998
Fold 1 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 2 - Accuracy: 0.996, Precision: 0.991, Recall: 0.998, F-measure: 0.995
Fold 3 - Accuracy: 0.996, Precision: 0.985, Recall: 0.998, F-measure: 0.991
Fold 4 - Accuracy: 0.996, Precision: 0.998, Recall: 0.991, F-measure: 0.995
Fold 5 - Accuracy: 0.993, Precision: 0.984, Recall: 0.997, F-measure: 0.990
Fold 6 - Accuracy: 0.937, Precision: 0.901, Recall: 0.972, F-measure: 0.927
Fold 7 - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F-measure: 1.0
Fold 8 - Accuracy: 0.998, Precision: 0.995, Recall: 0.999, F-measure: 0.997
Fold 9 - Accuracy: 0.998, Precision: 0.995, Recall: 0.999, F-measure: 0.997
Accuracy: 0.9915108295625927 on average and 0.01818389943705438 SD
Precision: 0.9850171738016691 on average and 0.028442557955791464 SD
Recall: 0.9956697791614939 on average and 0.007926533399194089 SD
F-measure: 0.9893840483153049 on a

In [18]:
classificador = KNN(42)
classificador.classificar(dataset, fold_method = "GroupKfold", n_splits_value = 10);

TypeError: classificar() missing 4 required positional arguments: 'search', 'params', 'score_op', and 'numero_cv'

In [ ]:
classificador = KNN(42)
classificador.classificar(dataset, fold_method = "StratifiedKfold", n_splits_value = 10);

In [ ]:
classificador = SVM(42)
classificador.classificar(dataset, fold_method = "GroupKfold", n_splits_value = 10);

In [ ]:
classificador = SVM(42)
classificador.classificar(dataset, fold_method = "StratifiedKfold", n_splits_value = 10);

In [ ]:
classificador = RandomForest(42)
classificador.classificar(dataset, fold_method = "GroupKfold", n_splits_value = 10);

In [ ]:
classificador = RandomForest(42)
classificador.classificar(dataset, fold_method = "StratifiedKfold", n_splits_value = 10);

In [ ]:
classificador = LR(42)
classificador.classificar(dataset, fold_method = "GroupKfold", n_splits_value = 10);

In [ ]:
classificador = LR(42)
classificador.classificar(dataset, fold_method = "StratifiedKfold", n_splits_value = 10);